# General overview

In [1]:
import json
import gzip
import pickle
from tqdm import tqdm
from collections import defaultdict
from helpers import bins_y_s

SRC = "/dlabdata1/manosphere-rad/"
df_sources = pd.read_csv(SRC + "sources_final_trimmed.csv")
cats_to_control = ["center", "left-center", "right-center", "left", "right", "NONE"]
cats_all = list(df_sources.Category.value_counts().keys())
cats = [cat for cat in cats_all if cat not in cats_to_control]
cats.append("Control")
cat_man = ['MGTOW', 'Incel', 'MRA', "PUA"]
cats.append('Manosphere')

In [2]:
with gzip.open(SRC+"channels_youtube.json.gz", 'rb') as f:
    channel_dict = json.load(f)

count = {b:{cat:0 for cat in cats} for b in bins_y_s}

def increment_cat_count_in_bin(b, category, timestamp_v):
    for lower, upper in b.keys():
        if upper > timestamp_v >= lower:
            b[(lower, upper)][category] += 1
            break

for cat in cats_all:
    final_cat = cat if cat in cats else "Control"
    for channel in df_sources.loc[df_sources["Category"] == cat, "Id"]:
        if channel in channel_dict:
            for v in channel_dict[channel]:
                increment_cat_count_in_bin(count, final_cat, v['timestamp'])

for k, v in count.items():
    manosphere = 0
    for cat in cat_man:
        manosphere += v[cat]
    count[k]['Manosphere'] = manosphere

# Checkpoint
with open("../helper_files/comments_stats.pickle", "wb") as f:
    pickle.dump(count, f)